In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Création du modèle
model = BayesianNetwork([('Account Age', 'Spending Patterns'),
                         ('Spending Patterns', 'Transaction Amount'),
                         ('Transaction Location', 'Transaction Amount'),
                         ('Transaction Amount', 'Fraud Likelihood'),
                         ('Transaction Location', 'Fraud Likelihood')])

# Définition des CPTs (Tables de probabilité conditionnelle)

# CPT pour 'Account Age'
cpd_account_age = TabularCPD(variable='Account Age', variable_card=3, 
                             values=[[0.4], [0.4], [0.2]])  # Proba: nouveau, moyen, ancien

# CPT pour 'Spending Patterns', dépend de 'Account Age'
cpd_spending_patterns = TabularCPD(variable='Spending Patterns', variable_card=2,
                                   values=[[0.7, 0.3, 0.2],  # normales, anormales
                                           [0.3, 0.7, 0.8]], 
                                   evidence=['Account Age'],
                                   evidence_card=[3])

# CPT pour 'Transaction Amount'
cpd_transaction_amount = TabularCPD(variable='Transaction Amount', variable_card=3, 
                                    values=[[0.4, 0.3, 0.1, 0.2],  # faible, moyen, élevé
                                            [0.3, 0.5, 0.2, 0.4],
                                            [0.3, 0.2, 0.7, 0.4]],
                                    evidence=['Spending Patterns', 'Transaction Location'], 
                                    evidence_card=[2, 2])

# CPT pour 'Transaction Location'
cpd_transaction_location = TabularCPD(variable='Transaction Location', variable_card=2, 
                                      values=[[0.9], [0.1]])  # Proba: locale, étrangère

# CPT pour 'Fraud Likelihood'
cpd_fraud_likelihood = TabularCPD(variable='Fraud Likelihood', variable_card=3, 
                                  values=[[0.7, 0.6, 0.3, 0.2, 0.1, 0.3],  # faible, modérée, élevée
                                          [0.2, 0.3, 0.4, 0.4, 0.6, 0.5],
                                          [0.1, 0.1, 0.3, 0.4, 0.3, 0.2]],
                                  evidence=['Transaction Amount', 'Transaction Location'], 
                                  evidence_card=[3, 2])

# Ajouter les CPTs au modèle
model.add_cpds(cpd_account_age, cpd_spending_patterns, cpd_transaction_amount, 
               cpd_transaction_location, cpd_fraud_likelihood)

# Vérification du modèle
model.check_model()

# Inférence
inference = VariableElimination(model)

# Effectuer une inférence pour estimer la probabilité de fraude en fonction de certaines variables
result = inference.query(variables=['Fraud Likelihood'], evidence={'Account Age': 0, 'Spending Patterns': 1, 'Transaction Location': 1})

print(result)

+---------------------+-------------------------+
| Fraud Likelihood    |   phi(Fraud Likelihood) |
+=====================+=========================+
| Fraud Likelihood(0) |                  0.3667 |
+---------------------+-------------------------+
| Fraud Likelihood(1) |                  0.4000 |
+---------------------+-------------------------+
| Fraud Likelihood(2) |                  0.2333 |
+---------------------+-------------------------+
